<a href="https://colab.research.google.com/github/SzymonNowakowski/Machine-Learning-2024/blob/master/Lab12_nlp-introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 12 - Natural Language Processing - Introduction

### Author: Szymon Nowakowski


# Introduction
---------------


# Google Crowdsource Sentiment Dataset
-------------------------

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the GoEmotions dataset (by Google)
dataset = load_dataset("go_emotions")

# Convert to binary sentiment (positive/neutral/negative)
def map_to_sentiment(example):
    positive = {1, 2, 3, 20, 26}  # joy, amusement, excitement, admiration, approval
    negative = {4, 5, 6, 9, 11, 18, 24}  # anger, annoyance, disappointment, disgust, fear, sadness, disapproval
    neutral = {0, 27}  # neutral, realization (as a proxy)

    labels = set(example["labels"])
    if labels & positive:
        return "positive"
    elif labels & negative:
        return "negative"
    else:
        return "neutral"

# Apply sentiment mapping
dataset = dataset["train"].map(lambda x: {"sentiment": map_to_sentiment(x)})

# Convert to DataFrame for easier use
df = dataset.to_pandas()[["text", "sentiment"]]
print(df.sample(5))  # Show sample entries

# Now df is ready to use in the rest of your pipeline
